In [1]:
# -*- coding: utf-8 -*-
# 헤더 선언

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input,Dense, Lambda, Conv1D, Conv2DTranspose, LeakyReLU, Activation, Flatten, Reshape, BatchNormalization

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import utils
from utils import *
import featurizer
from featurizer import *

In [2]:
# Set the api key
mat_api_key = '2IgbGtrUqnFHQpV9'

sup_prop = ['formation_energy_per_atom']
semi_prop = ['band_gap']

# 3 elements, 20 sites
num_ele=3
num_sites = 20

BATCH_SIZE = 1024

In [3]:
# get the data and save it as csv file.
df, df_in =  get_data(mat_api_key, num_sites+1, 2,num_ele+1)
# df.to_csv('./data/df.csv')
# df_in.to_csv('./data/df_in.csv')
# df = pd.read_csv('./data/df.csv', index_col=0)
# df_in = pd.read_csv('./data/df_in.csv', index_col=0)

# Get information on elements.
elem_embedding_file = './files/atom_init.json'
with open(elem_embedding_file) as f:
    elem_embedding = json.load(f)
elem_embedding = {int(key): value for key, value in elem_embedding.items()}

# change the array size to (100, 92)
feat_cgcnn = []
for key,value in elem_embedding.items():
    feat_cgcnn.append(value)
feat_cgcnn = np.array(feat_cgcnn)
feat_cgcnn = feat_cgcnn.astype(np.float32)

  0%|          | 0/23951 [00:00<?, ?it/s]

C:\Users\vnvnr\Desktop\material\featurizer.py:52: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_in = pd.concat([df.iloc[:,:],df_m.loc[i,:]],1)


In [ ]:
# num_ele should be equal to most_ele-1
Crystal = crystal_represent(df,num_ele,num_sites)

# It took a long time to change to a crystal expression, so I made it into an npy file.
# np.save('./crystal_numpy/base', Crystal)
# Crystal = np.load('./crystal_numpy/base.npy')

--crystal_repersent start--



  0%|                                                                                | 5/23951 [00:00<08:47, 45.39it/s]C:\Users\vnvnr\Anaconda3\envs\material\lib\site-packages\pymatgen\io\cif.py:1116: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))

  2%|█▊                                                                            | 574/23951 [00:07<04:39, 83.75it/s]


  5%|███▋                                                                         | 1138/23951 [00:15<06:30, 58.43it/s]


  7%|█████▍                                                                       | 1701/23951 [00:23<04:42, 78.75it/s]


  9%|███████▎                                                                     | 2266/23951 [00:31<05:08, 70.27it/s]


...

 24%|██████████████████▋                                                          | 5809/23951 [01:27<05:27, 55.44it/s]


 26%|████████████████████▏                                                        | 6275/23951 [01:35<06:16, 46.90it/s]


 28%|█████████████████████▋                                                       | 6731/23951 [01:44<03:50, 74.69it/s]


 30%|███████████████████████▎                                                     | 7235/23951 [01:52<04:02, 69.04it/s]


 32%|████████████████████████▍                                                    | 7618/23951 [01:58<03:06, 87.42it/s]


In [ ]:
# preprocessing X
X= Crystal
X= pad(X, 2)
X = X.astype(np.float32)

# Convert the scale of X to (0, 1).
X, scaler_x = minmax(X)

In [ ]:
# preprocessing Y
# Y is a matrix of formation energy perporter, bandgap, and index
Y = df_in[sup_prop+semi_prop+['ind'] ].values
Y = Y.astype(np.float32)
sup_dim = len(sup_prop)
semi_sup_dim = len(semi_prop)
scaler_y_un = MinMaxScaler()
scaler_y_l = MinMaxScaler()
Y[:,:sup_dim] = scaler_y_un.fit_transform(Y[:,:sup_dim])
Y[:,sup_dim:semi_sup_dim+sup_dim] = scaler_y_l.fit_transform(Y[:,sup_dim:semi_sup_dim+sup_dim])

# Find the index of the matrix where the blanks are removed.
label_index = df_in[df_in['band_gap'] != 0]['ind'].values
label_index.shape

In [ ]:
# 디코더의 인풋을 형성하는 부분
D = Crystal[:, :103, 0]
D = np.concatenate((D, Crystal[:, 146:, 0]), axis=1)
D = np.concatenate((D, Crystal[:, :103, 1]), axis=1)
D = np.concatenate((D, Crystal[:, 146:, 1]), axis=1)
D = np.concatenate((D, Crystal[:, :103, 2]), axis=1)
D = np.concatenate((D, Crystal[:, 146:, 2]), axis=1)
D = D.astype(np.float32)

# d와 y를 합쳐 같이 나눈다.
d_y = np.concatenate((D, Y), axis=1)
d_y.shape

In [ ]:
# 데이터셋을 나눈다. 테스트 0.2
from sklearn.model_selection import train_test_split
X_train, X_test, d_y_train, d_y_test = train_test_split(X, d_y, test_size=0.2, random_state=21)
d_train = d_y_train[:, :585]
y_train = d_y_train[:, 585:]
d_test = d_y_test[:, :585]
y_test = d_y_test[:, 585:]

In [ ]:
# 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, d_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test, d_test))
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
# make vae model 
K.clear_session()

# 240, 60, 3
input_dim = X_train.shape[1]
channel_dim = X_train.shape[2]
sample_dim = y_train.shape[1]

latent_dim = 256
max_filter = 128
strides = [2,2,1]
kernel = [5,3,3]
# coefficient for KL, regression of ground state properties and regression of sparse properties
reg_kl = 2 
reg_sup = 10
reg_semi = 5

In [ ]:
x = Input(shape=(input_dim,channel_dim,))  # (240, 60,)
y = Input(shape =(sample_dim ,))  # (3,)
y_label = Input(shape =(semi_sup_dim ,))  # (1,)


# 데이터 타입을 변경
label_ind = tf.convert_to_tensor(label_index, dtype =tf.int64)
y1 = y[:,:-1]

# 라벨이 있는 것만 추려내는 역할
def get_idx(y):
    # 인덱스 부분을 받아서
    y_ind = y[:,-1]   
    # dtype을 바꾸고
    y_ind = tf.dtypes.cast(y_ind,tf.int64)
    # id가 동일한 부분들을 찾는다.
    com_ind= tf.sets.intersection(y_ind[None,:],label_ind[None,:])
    com_ind  = tf.sparse.to_dense(com_ind)
    com_ind = tf.squeeze(com_ind)
    com_ind = tf.reshape(com_ind,(tf.shape(com_ind)[0],1))
    semi_index = tf.where(tf.equal(y_ind,com_ind))[:,-1]
    return semi_index
# 특성2의 값에서 빈 것을 지운 인덱스, 나중에 학습할 때 사용
semi_index = Lambda(get_idx)(y)

In [ ]:
# encoder 생성
en0 = Conv1D(max_filter//4,kernel[0],strides= strides[0], padding='SAME')(x)
en0 = BatchNormalization()(en0)
en0 = LeakyReLU(0.2)(en0)

en1 = Conv1D(max_filter//2,kernel[1],strides=strides[1], padding='SAME')(en0)
en1 = BatchNormalization()(en1)
en1 = LeakyReLU(0.2)(en1)

en2 = Conv1D(max_filter,kernel[2], strides=strides[2],padding='SAME')(en1)
en2 = BatchNormalization()(en2)
en2 = LeakyReLU(0.2)(en2)

en3 = Flatten()(en2)
en4 = Dense(1024, activation='relu')(en3)

z_mean = Dense(latent_dim, activation = 'linear')(en4)
z_log_var = Dense(latent_dim, activation = 'linear')(en4)

encoder_kl = Model(x, [z_mean, z_log_var])
    
epsilon_std =1

# encoder 지난 평균과 분산으로 레이턴시 상에서 샘플링
def sampling(args):
    z_mean, z_log_var = args
    # 표준 정규 분포를 따르는 엡실론 구함
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=epsilon_std)
    # 구한 엡실론을 이용하여 샘플링
    return z_mean + K.exp(0.5*z_log_var/2) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean,z_log_var])
encoder_ = Model(x,z)
encoder_.summary()
# encoder_kl.summary()

In [ ]:
# 특성 예측을 구성하는 부분
de = Input((latent_dim,))
de0 = Activation('relu')(de)
de1_un = Dense(128,activation = "relu")(de0)
de1_un =Dense(32,activation = "relu")(de1_un)
de1_l = Dense(128,activation = "relu")(de0)
de1_l =Dense(32,activation = "relu")(de1_l)
# 256->128->32->1 로 하나의 특성에 대해 값을 출력한다. 
y_predict_sup = Dense(sup_dim, activation ='sigmoid')(de1_un)
y_predict_semi = Dense(semi_sup_dim, activation ='sigmoid')(de1_l)
# semi인 경우, 빈 값이 있으므로 이에대한 처리를 해준다.
# y_predict_semi_after = tf.gather(y_predict_semi,semi_index ,axis=0)
y_label = tf.gather(y,semi_index ,axis=0)
# 특성을 추출하는 모델을 만든다.
pred_model_sup = Model(de, y_predict_sup)
pred_model_semi = Model(de, y_predict_semi)
pred_model_sup.summary()
pred_model_semi.summary()

In [ ]:
# 디코더 생성
map_size = K.int_shape(encoder_.layers[-6].output)[1]
z_in = Input(shape=(latent_dim,))
z1 = Dense(max_filter*map_size,activation='relu')(z_in)
z1 = Reshape((map_size,1,max_filter))(z1)
z1 = BatchNormalization()(z1)

z2 =  Conv2DTranspose( max_filter//2, (kernel[2],1), strides=(strides[2],1),padding='SAME')(z1)
z2 = BatchNormalization()(z2)
z2 = Activation('relu')(z2)

z3 = Conv2DTranspose(max_filter//4, (kernel[1],1), strides=(strides[1],1),padding='SAME')(z2)
z3 = BatchNormalization()(z3)
z3 = Activation('relu')(z3)

z4 = Conv2DTranspose(channel_dim, (kernel[0],1), strides=(strides[0],1),padding='SAME')(z3)
decoded_x = Activation('sigmoid')(z4)
decoded_x = tf.keras.layers.Reshape((240, 60))(decoded_x)
decoder_ = Model(z_in,decoded_x)
decoded_x = decoder_(z)
decoder_.summary()

In [ ]:
cvae = Model(inputs= x,outputs= decoded_x)
# cvae.summary()

In [ ]:
# fc2층 생성
f_in = Input(shape=(587,))
f1 = Dense(512, activation = "relu")(f_in)
f2 = Dense(384, activation = "relu")(f1)
f_out = Dense(256, activation="linear")(f2)
fc2 = Model(f_in, f_out)
fc2.summary()

In [ ]:
def vae_loss_function(x, decoded_x): 
    xent_loss = K.sum(K.square(x[:,:,:]- decoded_x[:,:,:]))
    return xent_loss

def kl_loss_function(z_mean, z_log_var):
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
    return kl_loss

def property_loss_function(labels, predicted_label):
    prop_loss = K.sum(K.square(labels - predicted_label))
    return prop_loss

def fc2_loss_function(latent, fc2_latent):
    fc2_loss = K.sum(K.square(latent[:, :] - fc2_latent[:, :]))
    return K.mean(fc2_loss)

def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true+1e-12), np.array(y_pred+1e-12)
    return np.mean(np.abs((y_true - y_pred) / (y_true))) * 100

In [ ]:
optimizer = optimizers.RMSprop(learning_rate=8e-4)

# validation loss가 4번동안 감소하지 않을시 lr을 0.3배
class lr_scheduler:
    def __init__(self, factor, patience, min_lr, optimizers):
        self.factor = factor
        self.patience = patience + 1
        self.min_lr = min_lr
        self.optimizers = optimizers
        
    def make_schedule(self, loss):
        if self.optimizers.learning_rate <= self.min_lr:
            return
        if len(loss) < self.patience + 1:
            return

        recent_loss = loss[-(self.patience):]
        count = 0
        for loss_value in recent_loss:
            if loss_value > loss[-(self.patience+1)]:
                count += 1
        if count == self.patience:
            lr = self.optimizers.learning_rate
            self.optimizers.learning_rate.assign(lr*self.factor)
            if self.optimizers.learning_rate < self.min_lr:
                self.optimizers.learning_rate.assign(self.min_lr)
        return
            
        
scheduler = lr_scheduler(factor=0.3, patience=4, min_lr=1e-6, optimizers=optimizer)

In [ ]:
# vae와 특성층 학습
@tf.function
def train_step1(images, labels):
    with tf.GradientTape() as vae_tape:
        z_mean, z_log_var = encoder_kl(images)
        z = sampling([z_mean, z_log_var])
        vae_images = decoder_(z)
        
        xent_loss = vae_loss_function(images, vae_images)
        kl_loss = kl_loss_function(z_mean, z_log_var)
        
        y_predict_sup = pred_model_sup(z_mean)
        pred_loss_sup = property_loss_function(labels[:, :sup_dim], y_predict_sup)
        
        y_predict_semi = pred_model_semi(z_mean)
        semi_index = Lambda(get_idx)(labels)
        y_predict_semi_after = tf.gather(y_predict_semi,semi_index ,axis=0)
        labels_after = tf.gather(labels,semi_index ,axis=0)
        pred_loss_semi = property_loss_function(labels_after[:, sup_dim:semi_sup_dim+sup_dim], y_predict_semi_after)
        
        vae_loss = K.mean(xent_loss + reg_kl*kl_loss + reg_sup*pred_loss_sup + reg_semi*pred_loss_semi)
        
    vae_gradients = vae_tape.gradient(vae_loss, cvae.trainable_variables)
    optimizer.apply_gradients(zip(vae_gradients, cvae.trainable_variables))
    
    return vae_loss


# fc2층 학습
@tf.function
def train_step2(images, d_input):
    z_mean, z_log_var = encoder_kl(images)
    y_predict_sup = pred_model_sup(z_mean)
    y_predict_semi = pred_model_semi(z_mean)
    d_input = tf.concat([d_input, y_predict_sup], 1)
    d_input = tf.concat([d_input, y_predict_semi], 1)
    with tf.GradientTape() as fc2_tape:
        fc2_latent = fc2(d_input)
        fc2_loss = fc2_loss_function(z_mean, fc2_latent)
    fc2_gradients = fc2_tape.gradient(fc2_loss, fc2.trainable_variables)
    optimizer.apply_gradients(zip(fc2_gradients, fc2.trainable_variables))
        
    return fc2_loss

# val loss를 구하는 층
@tf.function
def test_step(images, labels):
    z_mean, z_log_var = encoder_kl(images)
    z = sampling([z_mean, z_log_var])
    vae_images = decoder_(z)
    
    xent_loss = vae_loss_function(images, vae_images)
    kl_loss = kl_loss_function(z_mean, z_log_var)

    y_predict_sup = pred_model_sup(z_mean)
    pred_loss_sup = property_loss_function(labels[:, :sup_dim], y_predict_sup)

    y_predict_semi = pred_model_semi(z_mean)
    semi_index = Lambda(get_idx)(labels)
    y_predict_semi_after = tf.gather(y_predict_semi,semi_index ,axis=0)
    labels_after = tf.gather(labels,semi_index ,axis=0)
    pred_loss_semi = property_loss_function(labels_after[:, sup_dim:semi_sup_dim+sup_dim], y_predict_semi_after)

    vae_loss = K.mean(xent_loss + reg_kl*kl_loss + reg_sup*pred_loss_sup + reg_semi*pred_loss_semi)
    
    return vae_loss

In [ ]:
EPOCHS = 500
s_time = time.time()
loss_list = []
val_loss_list = []
fc2_loss_list = []
vae_images_list = []
for epoch in range(EPOCHS):
    loss = 0
    fc2_loss = 0
    train_counts = 0
    for images, labels, d_input in tqdm(train_dataset):
        loss += train_step1(images, labels) * len(images)
        fc2_loss += train_step2(images, d_input) * len(images)
        train_counts += len(images)
    loss = loss / train_counts
    fc2_loss = fc2_loss / train_counts
    loss_list.append(loss.numpy())
    fc2_loss_list.append(fc2_loss.numpy())
    lr = K.get_value(optimizer.lr)
    
    val_loss = 0
    val_counts = 0
    for test_images, test_labels, test_d_input in test_dataset:
        val_loss += test_step(test_images, test_labels) * len(test_images)
        val_counts += len(test_images)
    val_loss = val_loss / val_counts
    val_loss_list.append(val_loss.numpy())
    scheduler.make_schedule(val_loss_list)

    print("[epochs:", epoch+1, "] | [train loss:", loss.numpy(), ', val loss:', val_loss.numpy(), '] | [fc2 loss:', fc2_loss.numpy(), '] | [lr:', lr, ']')
e_time = time.time()
print("train time:", e_time - s_time, 's')

In [ ]:
# 여기까지가 학습, 후로는 테스트 및 검증

In [ ]:
# VAE loss 에 대한 에폭에 따른 loss 그래프
plt.plot(np.arange(20, len(loss_list)), loss_list[20:])
plt.plot(np.arange(20, len(val_loss_list)), val_loss_list[20:])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train_loss', 'val_loss'])
plt.savefig('graph.png')
plt.show()

In [ ]:
# fc2 loss에 대한 에폭에 따른 loss 그래프
plt.plot(np.arange(0, len(fc2_loss_list)), fc2_loss_list[0:])
plt.xlabel('epoch')
plt.ylabel('fc2_loss')
plt.savefig('graph2.png')
plt.show()

In [ ]:
# 테스트 데이터를 이용해 테스트
vae_x = cvae.predict(X_test)
len1 = X_train.shape[1]

In [ ]:
plt.scatter(np.linspace(0,len1,len1),vae_x[1,:,2],color ='red',label ='VAE')
plt.scatter(np.linspace(0,len1,len1),X_test[1,:,2],label ='Original')

In [ ]:
# 결과로 나온 이미지의 스케일을 다시 되돌린다.
vae_x = inv_minmax(vae_x,scaler_x)
vae_x [vae_x<0.1] =0
#X_test1 = pad(X_test, 1)
# 테스트 데이터셋도 똑같이 스케일을 되돌린다.
X_test1 = inv_minmax(X_test,scaler_x)

In [ ]:
# 오류를 구하는 함수
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true+1e-12), np.array(y_pred+1e-12)
    return np.mean(np.abs((y_true - y_pred) / (y_true))) * 100

def MAE(y_true, y_pred):
    y_true, y_pred = np.array(y_true+1e-12), np.array(y_pred+1e-12)
    return np.mean(np.abs(y_true-y_pred))

In [ ]:
# abc에 해당하는 값
abc_test = X_test1[:,103,:3]
abc_vae = vae_x[:,103,:3]

# ang에 해당하는 값
ang_test = X_test1[:,104,:3]
ang_vae = vae_x[:,104,:3]

# bas에 해당하는 값
bas_test = X_test1[:,105:105+num_sites,:3]
bas_vae = vae_x[:,105:105+num_sites,:3]

# VAE의 테스트 데이터에 대한 오류
print('abc_PMAE %',round(MAPE(abc_test,abc_vae),3))
print('angle_PMAE %',round(MAPE(ang_test,ang_vae),3))
print('basis_MAE',round(metrics.mean_squared_error(bas_test[:,:,0],bas_vae[:,:,0]),5))

In [ ]:
# 구성 원소에 대한 재구성 확인
accu = []
vae_ele = []
X_ele = []
# 원소의 기호를 받아온다. 이는 103개이다.
Element= joblib.load('./files/element.pkl') 
# 103개의 원소가 있고 이를 원핫 벡터로 표현한다.(103, 103)
E_v = np_utils.to_categorical(np.arange(0,len(Element),1))
for i in range(num_ele):
    ele_v = np.argmax(vae_x[:,0:len(E_v),i],axis=1)
    ele_t = np.argmax(X_test1[:,0:len(E_v),i],axis=1)
    vae_ele.append(ele_v)
    X_ele.append(ele_t)
    accu1 =100* round(metrics.accuracy_score(ele_v,ele_t),3)
    accu.append(accu1)
print('Element accuracy %',accu)

In [ ]:
# 테스트 데이터를 이용해 특성 비교
z_mean, z_log_var = encoder_kl(X_test[:,:,:])
y_predict_sup = pred_model_sup(z_mean)
y_predict_semi = pred_model_semi(z_mean)
d_input = tf.concat([d_test[:, :], y_predict_sup], 1)
d_input = tf.concat([d_input, y_predict_semi], 1)
fc2_latent = fc2(d_input)

In [ ]:
# 잠재벡서 fc 층을 이용해 이미지 생성
vae_images = decoder_(fc2_latent)
temp = vae_images[0]
plt.imshow(temp)

In [ ]:
# 잠재벡터 fc 층을 이용해서 생성한 이미지에서 특성 예측
z_mean2, z_log_var2 = encoder_kl(vae_images[:,:,:])
y_predict_sup2 = pred_model_sup(z_mean2)
y_predict_semi2 = pred_model_semi(z_mean2)

In [ ]:
# 잠재벡터 fc 모듈을 이용해 생성한 이미지에서 입력 구성 원소가 제대로 반영되는지 확인
accu = []
vae_ele = []
X_ele = []
# 원소의 기호를 받아온다. 이는 103개이다.
Element= joblib.load('./files/element.pkl') 
# 103개의 원소가 있고 이를 원핫 벡터로 표현한다.(103, 103)
E_v = np_utils.to_categorical(np.arange(0,len(Element),1))
for i in range(num_ele):
    ele_v = np.argmax(vae_images[:,0:len(E_v),i],axis=1)
    ele_t = np.argmax(X_test[:,0:len(E_v),i],axis=1)
    vae_ele.append(ele_v)
    X_ele.append(ele_t)
    accu1 =100* round(metrics.accuracy_score(ele_v,ele_t),3)
    accu.append(accu1)
print('Element accuracy %',accu)

In [ ]:
recon_test_images = X_test
recon_test_labels = y_test
recon_test_d = d_test
print(recon_test_labels[:, 1].shape)
non_zero_index = np.nonzero(recon_test_labels[:, 1])[0]
recon_test_images = recon_test_images[non_zero_index]
recon_test_labels = recon_test_labels[non_zero_index]
recon_test_d = recon_test_d[non_zero_index]
recon_test_d_input = np.concatenate((recon_test_d, recon_test_labels[:, :2]), axis=1)
print(recon_test_d_input.shape)
recon_latent = fc2(recon_test_d_input)
recon_images = decoder_(recon_latent)

In [ ]:
# 잠재벡터 fc 모듈을 이용해 생성한 이미지에서 입력 구성 원소가 제대로 반영되는지 확인
accu = []
vae_ele = []
X_ele = []
# 원소의 기호를 받아온다. 이는 103개이다.
Element= joblib.load('./files/element.pkl') 
# 103개의 원소가 있고 이를 원핫 벡터로 표현한다.(103, 103)
E_v = np_utils.to_categorical(np.arange(0,len(Element),1))
for i in range(num_ele):
    ele_v = np.argmax(recon_images[:,0:len(E_v),i],axis=1)
    ele_t = np.argmax(recon_test_images[:,0:len(E_v),i],axis=1)
    vae_ele.append(ele_v)
    X_ele.append(ele_t)
    accu1 =100* round(metrics.accuracy_score(ele_v,ele_t),3)
    accu.append(accu1)
print('Element accuracy %',accu)

In [ ]:
# 잠재벡터 fc를 통해 재구성한 이미지와 테스트 데이터 비교
# abc에 해당하는 값
abc_test = recon_test_images[:,103,:3]
abc_vae = recon_images[:,103,:3]

# ang에 해당하는 값
ang_test = recon_test_images[:,104,:3]
ang_vae = recon_images[:,104,:3]

# bas에 해당하는 값
bas_test = recon_test_images[:,105:105+num_sites,:3]
bas_vae = recon_images[:,105:105+num_sites,:3]

print('abc_PMAE %',round(MAPE(abc_test,abc_vae),3))
print('angle_PMAE %',round(MAPE(ang_test,ang_vae),3))
print('basis_MAE',round(metrics.mean_squared_error(bas_test[:,:,0],bas_vae[:,:,0]),5))

In [ ]:
# vae의 오류율을 만족하는 소재를 찾아본다.
count = 0
for i in range(len(abc_test)):
    mape_abc = MAPE(abc_test[i], abc_vae[i])
    mape_ang = MAPE(ang_test[i], ang_vae[i])
    if mape_abc < 10 and mape_ang < 5:
        count += 1
print(count)

In [ ]:
a1 = scaler_y_un.inverse_transform(y_test[:, 0].reshape((len(y_test[:, 0]), 1)))
a2 = scaler_y_un.inverse_transform(y_predict_sup.numpy())
a3 = scaler_y_un.inverse_transform(y_predict_sup2.numpy())

In [ ]:
# test 데이터의 주 특성에 대한 mean absolute error
# 첫 번째는 vae의 특성 예측에 대한 오류이고, 두 번째는 잠재벡터 fc 모듈을 이용해 생성한 이미지의 특성 오류이다. 
print('test sup')
print(round(MAE(a1, a2), 6))
print(round(MAE(a2, a3), 6))

In [ ]:
# test semi MAE를 계산하는 부분
b1 = []
b2 = []
b3 = []
count = 0
for i in range(len(y_test[:, 1])):
    if y_test[:, 1][i] != 0:
        b1.append(y_test[:, 1][i])
        b2.append(y_predict_semi[i])
        b3.append(y_predict_semi2[i])

In [ ]:
b1 = np.array(b1).reshape((len(b1), 1))
b2 = np.array(b2).reshape((len(b2), 1))
b3 = np.array(b3).reshape((len(b3), 1))

In [ ]:
b1 = scaler_y_l.inverse_transform(b1)
b2 = scaler_y_l.inverse_transform(b2)
b3 = scaler_y_l.inverse_transform(b3)

In [ ]:
# 테스트 데이터의 부 특성에 대한 MAE
# 첫 번째는 vae의 특성 예측에 대한 오류이고, 두 번째는 잠재벡터 fc 모듈을 이용해 생성한 이미지의 특성 오류이다.
print('test semi')
print(round(MAE(b1, b2), 6))
print(round(MAE(b2, b3), 6))